In [1]:
!pip install graphdatascience

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 789.5 kB/s eta 0:00:000:00:01m eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 602.2 kB/s eta 0:00:0031m12.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.3 MB/s eta 0:00:00m eta 0:00:010:01:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 540.7 kB/s eta 0:00:001m1.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 2.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 1.1 MB/s eta 0:00:000:00:01m eta 0:00:01
  Created wheel for neo

In [2]:
from graphdatascience import GraphDataScience

/home/pkedia/miniconda3/envs/cfde/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_AUTH = (
    "neo4j",
    "Hello@001"
)
NEO4J_DB = "neo4j"

gds = GraphDataScience(NEO4J_URI, auth = NEO4J_AUTH, database = NEO4J_DB)

In [17]:
from graphdatascience.server_version.server_version import ServerVersion
assert gds.server_version() >= ServerVersion(2,3,0)

In [1]:
"""
Required dataframe type for nodes and relationships

nodes = pandas.DataFrame(
    {
        "nodeId": [0, 1, 2, 3],
        "labels":  ["A", "B", "C", "A"],
        "prop1": [42, 1337, 8, 0],
        "otherProperty": [0.1, 0.2, 0.3, 0.4]
    }
)

relationships = pandas.DataFrame(
    {
        "sourceNodeId": [0, 1, 2, 3],
        "targetNodeId": [1, 2, 3, 0],
        "relationshipType": ["REL", "REL", "REL", "REL"],
        "weight": [0.0, 0.0, 0.1, 42.0]
    }
)

"""

'\nRequired dataframe type for nodes and relationships\n\nnodes = pandas.DataFrame(\n    {\n        "nodeId": [0, 1, 2, 3],\n        "labels":  ["A", "B", "C", "A"],\n        "prop1": [42, 1337, 8, 0],\n        "otherProperty": [0.1, 0.2, 0.3, 0.4]\n    }\n)\n\nrelationships = pandas.DataFrame(\n    {\n        "sourceNodeId": [0, 1, 2, 3],\n        "targetNodeId": [1, 2, 3, 0],\n        "relationshipType": ["REL", "REL", "REL", "REL"],\n        "weight": [0.0, 0.0, 0.1, 42.0]\n    }\n)\n\n'

In [44]:
GRAPH_NAME = "gene-gnn"
PIPELINE_NAME = "gene-pipeline"

In [60]:

if gds.graph.exists(GRAPH_NAME).exists:
    G = gds.graph.get(GRAPH_NAME)
else:
    G, _ = gds.graph.project(
        "gene-gnn",
        {"Gene": {"properties": ["node_code"]}},
        ["associated_with", "gene_encodes_gene_product"],
        readConcurrency = 4
    )

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.UnsupportedOperationException: Loading of values of type String is currently not supported}

In [51]:
print(G.node_labels(), G.relationship_types(), G.node_properties())

['Gene', 'EFO'] ['gene_encodes_gene_product', 'associated_with'] Gene    []
EFO     []
dtype: object


In [52]:
if gds.pipeline.exists(PIPELINE_NAME).exists:
    pipe = gds.pipeline.get(PIPELINE_NAME)
else:
    pipe, _ = gds.beta.pipeline.nodeClassification.create(PIPELINE_NAME)
    
_ = pipe.configureSplit(testFraction = 0.796)

In [53]:
_ = pipe.addNodeProperty(
    "beta.hashgnn",
    mutateProperty="embedding",
    iterations = 4, 
    heterogeneous = True, 
    embeddingDensity = 512, 
    neighborInfluence = 0.7,
    featureProperties = ["lower_bound", "upper_bound", "value"],
    randomSeed = 42,
    contextNodeLabels = G.node_labels()
)

In [54]:
_ = pipe.selectFeatures("embedding")

In [55]:
_ = pipe.addLogisticRegression(penalty = (0.1, 1.0), maxEpochs = 1000, patience = 5, tolerance = 0.0001, learningRate = 0.01)
_ = pipe.addRandomForest(minSplitSize = (2, 100), criterion = "ENTROPY")

In [56]:
model, _ = pipe.train(
    G,
    modelName = "gene-model",
    targetNodeLabels = ["Gene"],
    targetProperty = "lower_bound",
    metrics = ["F1_MACRO"],
    randomSeed = 42
)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.beta.pipeline.nodeClassification.train`: Caused by: java.lang.IllegalArgumentException: Target property `lower_bound` not found in graph with node properties: []}